In [ ]:
import MDAnalysis as mda
from MDAnalysis import transformations as trans

import numpy as np

In [ ]:
# load trajectory
u = mda.Universe("../input_data/interface.psf",
                 "../input_data/trajectory.xyz")
dim = [16.869, 16.869, 41.478, 90, 90, 120]
transform = trans.boxdimensions.set_dimensions(dim)
u.trajectory.add_transformations(transform)
u.transfer_to_memory(step=100)

# MDA v.s. m-MDA (serial version)

## MDA

In [ ]:
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis

In [ ]:
hbonds = HydrogenBondAnalysis(
    universe=u,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False
)
hbonds.run()
print(hbonds.results.hbonds.shape)
np.save("./mda_all.npy", hbonds.results.hbonds)

## m-MDA

In [ ]:
from watdyn.m_mda import PartialHBAnalysis

In [ ]:
hbonds = PartialHBAnalysis(
    universe=u,
    region=None,
    surf_ids=None,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False
)
hbonds.run()
print(hbonds.results.hbonds.shape)
np.save("./m-mda_all.npy", hbonds.results.hbonds)

## result check

In [ ]:
data_mda = np.load("./mda_all.npy")
data_mmda = np.load("./m-mda_all.npy")
# check if two array are identical
mask = (data_mda - data_mmda) == 0
print(False in mask)
# if False, all element in mask is True

# serial v.s. parallel

In [ ]:
from watdyn.m_mda import PartialHBAnalysis
from watdyn.parallel import parallel_exec

In [ ]:
hbonds = PartialHBAnalysis(
    universe=u,
    region=None,
    surf_ids=None,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False
)
parallel_exec(hbonds.run,0,10000,1,4)
print(hbonds.results.hbonds.shape)
#np.save("./m-mda_all_parallel.npy", hbonds.results.hbonds)

In [ ]:
data_mmda = np.load("./m-mda_all.npy")
data_mmda_p = np.load("./m-mda_all_parallel.npy")
# check if two array are identical
mask = (data_mmda - data_mmda_p) == 0
print(False in mask)
# if False, all element in mask is True

# partial HB analysis

In [ ]:
hbonds = PartialHBAnalysis(universe=u,
                           region=[2.7, 4.5],
                           surf_ids=[np.arange(871, 906),
                                     np.arange(691, 726)],
                           donors_sel=None,
                           hydrogens_sel="name H",
                           acceptors_sel="name O",
                           d_a_cutoff=3.0,
                           d_h_a_angle_cutoff=150,
                           update_selections=False)
hbonds.run(start=10, stop=30, step=2)
print(hbonds.results.hbonds.shape)
#np.save("./m-mda_interface_serial.npy", hbonds.results.hbonds)

In [ ]:
print(hbonds.start, hbonds.stop, hbonds.step)
print(hbonds.frames)

In [ ]:
hbonds = PartialHBAnalysis(universe=u,
                           region=[2.7, 4.5],
                           surf_ids=[np.arange(871, 906),
                                     np.arange(691, 726)],
                           donors_sel=None,
                           hydrogens_sel="name H",
                           acceptors_sel="name O",
                           d_a_cutoff=3.0,
                           d_h_a_angle_cutoff=150,
                           update_selections=False)
parallel_exec(hbonds.run,0,100,1,4)
print(hbonds.results.hbonds.shape)
#np.save("./m-mda_interface_parallel.npy", hbonds.results.hbonds)

In [ ]:
data_s = np.load("./m-mda_interface_serial.npy")
data_p = np.load("./m-mda_interface_parallel.npy")
# check if two array are identical
mask = (data_s - data_p) == 0
print(False in mask)
# if False, all element in mask is True